In [2]:
.libPaths('/cellar/users/e5silva/anaconda3/envs/r3_3/lib/R/library/') 
library('rentrez')

In [3]:
# import list
not_there <- read.table('pc_not_there.txt', header=FALSE, stringsAsFactors=FALSE)
dim(not_there)[1]
not_there[1:10,1]

[1] 949

[1] "LINC00238"     "ZNF295"        "CHURC1-FNTB"   "C2orf82"      
 [5] "P2RX5-TAX1BP3" "LOH12CR1"      "C9orf156"      "DPCR1"        
 [9] "AC138811.2"    "OPN1MW3"

In [ ]:
# testing getting out one record

# first look up ENTREZ ID 
r_search <- entrez_search(db='gene', term=paste(not_there[2,1], 'AND human[ORGN]'))$ids
r_search$ids

r_search <- entrez_summary(db='gene', id=r_search$ids[2])
test <- list()
test <- c(test, r_search$otheraliases, r_search$name)
test

In [4]:
# if have one ID, add to table 
now_there <- data.frame(Old_ID=character(), New_ID=character(), stringsAsFactors=FALSE)

for (i in c(1:949)) {
    
    # entrez search to find IDs associated with the supplied gene name in humans
    r_search <- entrez_search(db='gene', term=paste(not_there[i,1], "AND Homo sapiens[ORGN]"))$ids
    
    # if successfully find ONE id, then search for the given HGNC ID and add to table
    if (length(r_search) == 1) {
        
        tryCatch({
        
        r_search <- entrez_summary(db='gene', id=r_search)$name
        
        # add values to table
        now_there[i,1] <- not_there[i,1]
        now_there[i,2] <- r_search
            
            }, error = function(i) {return(NA)})
        
    }
    
    # if more than one ID
    if (length(r_search) > 1) {
        
        # repeat search
        r_search <- ''
        r_search <- entrez_search(db='gene', term=paste(not_there[i,1], "AND Homo sapiens[ORGN]"))
         
        ids <- ''
        ids <- r_search$ids
        
        # loop through IDs until find name or alias that matches query, then keep that name 
        test <- list()
        
        j=0
        while ((j < length(ids))==TRUE & (toupper(not_there[i,1]) %in% toupper(test))==FALSE) {
            
        tryCatch({
                        
            # loop through IDs, add name and aliases
            test <- list()
            j = j+1
            new_search <- ''
            new_search <- entrez_summary(db='gene', id=ids[j])
            test <- c(test, trimws(unlist(strsplit(new_search$name, ','))), trimws(unlist(strsplit(new_search$otheraliases,','))))            
            }, error = function(i) {return(NA)})
            
        }            

        
        # add values to table
        now_there[i,1] <- not_there[i,1]
        now_there[i,2] <- new_search$name
    } 

}


In [5]:
# check dims
dim(now_there)

[1] 949   2

In [6]:
# check table
now_there[1:10,1:2]

Old_ID,New_ID
LINC00238,CCDC196
ZNF295,ZBTB21
CHURC1-FNTB,CHURC1-FNTB
C2orf82,SNORC
P2RX5-TAX1BP3,P2RX5-TAX1BP3
LOH12CR1,BORCS5
C9orf156,TRMO
DPCR1,MUCL3
NA,NA
OPN1MW3,OPN1MW3


In [7]:
# save
write.table(now_there, file = "pc_now_there.csv", row.names=FALSE, col.names=FALSE, na="")

In [ ]:
i
j
length(ids)
now_there[i,1]
ids
ids[j]

new_search